<a href="https://colab.research.google.com/github/carlogalli/colab-gpu/blob/main/jmp_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you run the first cell (the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.7* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU).

In [1]:
VERSION   # print Julia version

v"1.7.2"

In [ ]:
@time begin
  using Pkg
  Pkg.add(["CUDA", "Random", "Distributions", "Printf", "BenchmarkTools", "PyPlot", "PrettyTables"]);
  #ENV["JULIA_CUDA_USE_BINARYBUILDER"] = false
  using Random, Distributions, CUDA, Printf, BenchmarkTools, PyPlot, PrettyTables;
end
# with the falseENV option it takes 117.357304 seconds (35.94 M allocations: 2.301 GiB, 1.29% gc time, 11.71% compilation time)
# without the falseENV option it takes  110.114893 seconds (35.54 M allocations: 2.282 GiB, 1.33% gc time, 11.66% compilation time)

function print_gpu_properties()

    for (i,device) in enumerate(CUDA.devices())
        println("*** General properties for device $i ***")
        name = CUDA.name(device)
        println("Device name: $name")
        major = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR)
        minor = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR)
        println("Compute capabilities: $major.$minor")
        clock_rate = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_CLOCK_RATE)
        println("Clock rate: $clock_rate")
        device_overlap = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_GPU_OVERLAP)
        print("Device copy overlap: ")
        println(device_overlap > 0 ? "enabled" : "disabled")
        kernel_exec_timeout = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_KERNEL_EXEC_TIMEOUT)
        print("Kernel execution timeout: ")
        println(kernel_exec_timeout > 0 ? "enabled" : "disabled")
        # a = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_BLOCK_DIM_X)
        # d = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_GRID_DIM_X)       
        a = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MULTIPROCESSOR_COUNT)
        println("Number of multiprocessors: $a")
        b = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_BLOCKS_PER_MULTIPROCESSOR)
        println("Max blocks per MP: $b")
        c = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)        
        println("Max threads per block: $c")
        
        println([a b c a*b*c])
    end
end

print_gpu_properties()

#Pkg.add("CpuId")
#using CpuId
#cpuinfo()

# JMP stuff
- Model structure
- Model constructor
- tauchen function
- `ydef_fn`
- my dot product
- my Brent

In [290]:
CUDA.allowscalar(false)

struct Model_GPU
    # Model Parameters
    β::Float32      # df govt
    β_hh::Float32   # df hh
    σ::Float32      # risk aversion
    α_m::Float32    # MIU scaling factor
    η::Float32      # MIU curvature
    γ::Float32      # govt spending utility curvature
    g_lb::Float32   # govt spending lower bound
    α_g::Float32    # govt spending weight
    r::Float32      # intnl risk-free rate
    ρ::Float32      # income process persistence
    σ_ϵ::Float32    # income process error standard deviation
    θ::Float32      # ree-enter probability
    h::Float32      # haircut on debt
    d0::Float32     # linear coefficient on default costadd C
    d1::Float32     # quadratic coefficient on default cost
    ρ_B::Float32    # coefficient of Gumbel debt taste shocks
    ρ_μ::Float32    # coefficient of Gumbel mu_D taste shocks
    ρ_δ::Float32    # coefficient of Gumbel enforcement shocks
    λ::Float32      # debt maturity
    κ::Float32      # debt coupon

    ny::Int32
    nB::Int32
    T_sim::Int32
    T_discard_sim::Int32

    Bgrid_lb::Float32
    Bgrid_ub::Float32
end

function Model_GPU(;
    β=.83, β_hh=.99, σ=2., α_m=2e-5, η=3., γ=2., g_lb=0., α_g=0.074, r=0.00598,
    ρ=0.9293, σ_ϵ=0.0115, θ=0.282, h=0.37, d0=-0.4, d1=0.44, 
    ρ_B=1e-3, ρ_μ=1e-3, ρ_δ=1e-3, λ=0.0465, κ=0.,
    ny=51, nB=150, T_sim=10^3, T_discard_sim=100, Bgrid_lb=1e-2, Bgrid_ub=2.5
    )

    N = ny*nB

    return Model_GPU(β, β_hh, σ, α_m, η, γ, g_lb, α_g, r, ρ, σ_ϵ, θ, h, d0, d1, ρ_B, ρ_μ, ρ_δ, λ, κ, ny, nB, T_sim, T_discard_sim, Bgrid_lb, Bgrid_ub)
end

function tauchen_carlo(N::Int32, ρ::Float32, σ_ϵ::Float32; μ::Real=0, n_std::Real=4)

    # process (x_t above) standard deviation (std is ϵ's standard deviation)
    σ_y = σ_ϵ/sqrt(1-ρ^2)
    grid_bar = n_std * σ_y      # grid bounds
    grid = collect(Float32, range(μ-grid_bar, stop=μ+grid_bar, length=N))
    dif = grid[2]-grid[1]
  
    # Get transition probabilities
    P = zeros(Float32, N, N)
    
    # return nothing

    for row in 1:N
        
        # do endpoints first
        P[row, 1] = cdf(Normal(0., 1.), (grid[1]+dif/2 -(1-ρ)*μ -ρ*grid[row])/σ_ϵ )
        P[row, N] = 1-cdf(Normal(0., 1.), (grid[N]-dif/2 -(1-ρ)*μ -ρ*grid[row])/σ_ϵ )

        # middle columns
        for col in 2:N-1
            P[row, col] = cdf(Normal(0., 1.), (grid[col]+dif/2 -(1-ρ)*μ -ρ*grid[row])/σ_ϵ ) - cdf(Normal(0., 1.), (grid[col]-dif/2 -(1-ρ)*μ -ρ*grid[row])/σ_ϵ )
        end
    end

    # normalize
    sums = sum(P, dims=2)    

    P = @. P/sums
    
    if maximum(abs.(sum(P, dims=2).-1)) > 1e-6
        error("Matrix rows must sum up to 1!")
    end
  
    return grid, P
end

function ydef_fn(m::Model_GPU, y::Float32)
    return y - max(0, m.d0*y + m.d1*y^2)
end

function my_dot_product(xvec, yvec)
    temp = Float32(0)
    for i in 1:lastindex(xvec)
        temp += xvec[i]*yvec[i]
    end
    return temp
end

function my_brent(
    f::Function , x_lower::Float32, x_upper::Float32;
    rel_tol::Float32 = sqrt(eps(Float32)),
    abs_tol::Float32 = eps(Float32),
    iterations::Int32 = Int32(1000),
    my_trace::Bool = false)


    if x_lower > x_upper
        error("x_lower must be less than x_upper")
    end

    golden_ratio = Float32(1)/2 * (3 - sqrt(Float32(5.0)))

    new_minimizer = x_lower + golden_ratio*(x_upper-x_lower)
    new_minimum = f(new_minimizer)
    best_bound = "initial"
    f_calls = 1 # Number of calls to f
    step = zero(Float32)
    old_step = zero(Float32)

    old_minimizer = new_minimizer
    old_old_minimizer = new_minimizer

    old_minimum = new_minimum
    old_old_minimum = new_minimum

    iteration = 0
    converged = false
    stopped_by_callback = false

    while iteration < iterations && !stopped_by_callback

        p = zero(Float32)
        q = zero(Float32)

        x_tol = rel_tol * abs(new_minimizer) + abs_tol

        x_midpoint = (x_upper+x_lower)/2

        if abs(new_minimizer - x_midpoint) <= 2*x_tol - (x_upper-x_lower)/2
            converged = true
            break
        end

        iteration += 1

        if abs(old_step) > x_tol
            # Compute parabola interpolation
            # new_minimizer + p/q is the optimum of the parabola
            # Also, q is guaranteed to be positive

            r = (new_minimizer - old_minimizer) * (new_minimum - old_old_minimum)
            q = (new_minimizer - old_old_minimizer) * (new_minimum - old_minimum)
            p = (new_minimizer - old_old_minimizer) * q - (new_minimizer - old_minimizer) * r
            q = 2(q - r)

            if q > 0
                p = -p
            else
                q = -q
            end
        end

        if abs(p) < abs(q*old_step/2) && p < q*(x_upper-new_minimizer) && p < q*(new_minimizer-x_lower)
            old_step = step
            step = p/q

            # The function must not be evaluated too close to x_upper or x_lower
            x_temp = new_minimizer + step
            if ((x_temp - x_lower) < 2*x_tol || (x_upper - x_temp) < 2*x_tol)
                step = (new_minimizer < x_midpoint) ? x_tol : -x_tol
            end
        else
            old_step = (new_minimizer < x_midpoint) ? x_upper - new_minimizer : x_lower - new_minimizer
            step = golden_ratio * old_step
        end

        # The function must not be evaluated too close to new_minimizer
        if abs(step) >= x_tol
            new_x = new_minimizer + step
        else
            new_x = new_minimizer + ((step > 0) ? x_tol : -x_tol)
        end

        new_f = f(new_x)
        f_calls += 1

        if new_f < new_minimum
            if new_x < new_minimizer
                x_upper = new_minimizer
                best_bound = "upper"
            else
                x_lower = new_minimizer
                best_bound = "lower"
            end
            old_old_minimizer = old_minimizer
            old_old_minimum = old_minimum
            old_minimizer = new_minimizer
            old_minimum = new_minimum
            new_minimizer = new_x
            new_minimum = new_f
        else
            if new_x < new_minimizer
                x_lower = new_x
            else
                x_upper = new_x
            end
            if new_f <= old_minimum || old_minimizer == new_minimizer
                old_old_minimizer = old_minimizer
                old_old_minimum = old_minimum
                old_minimizer = new_x
                old_minimum = new_f
            elseif new_f <= old_old_minimum || old_old_minimizer == new_minimizer || old_old_minimizer == old_minimizer
                old_old_minimizer = new_x
                old_old_minimum = new_f
            end
        end
    end

    if my_trace
        out = iteration, iteration == iterations, converged, rel_tol, abs_tol, f_calls
        return new_minimizer, new_minimum, out
    else
        return new_minimizer, new_minimum
    end
end

function model_init!(m, grids, arrays_rep, arrays_def)
    Bgrid, ygrid, ~ = grids
    vr, c_rep, rb_rep, Bprime_rep, ~, ~, q_rep, ~ = arrays_rep
    vd, c_def, rb_def, Bprime_def, μ_def, ~, q_def, ~ = arrays_def

    B0 = (blockIdx().x-1)*blockDim().x + threadIdx().x
    y0 = (blockIdx().y-1)*blockDim().y + threadIdx().y
    # tid_x, tid_y, bid_x, bid_y = threadIdx().x, threadIdx().y, blockIdx().x, blockIdx().y
    # @cuprintln("y0 $y0, B0 $B0, tid_x $tid_x, tid_y $tid_y, bid_x $bid_x, bid_y $bid_y")

    if B0 <= lastindex(Bgrid) && y0 <= lastindex(ygrid)
        # #= Repayment: in the last period, max U(c,rb,g) s.t. c=y-̃B*rb =#
        rb_rep[B0,y0] = 0.5 * ygrid[y0]/Bgrid[B0]
        c_rep[B0,y0] = ygrid[y0]-Bgrid[B0]*rb_rep[B0,y0]
        vr[B0,y0] = (c_rep[B0,y0]^(1-m.σ))/(1-m.σ) + m.α_m*(rb_rep[B0,y0]^(1-m.η))/(1-m.η)
        Bprime_rep[B0,y0] = 0.
        q_rep[B0, y0] = 1.
        
        # #= Default: in the last period, max U(c,rb,g) s.t. c=y_def-̃B*(1-h)*rb =#
        yd = ydef_fn(m,ygrid[y0])
        rb_def[B0,y0] = 0.5*yd/(Bgrid[B0]*(1-m.h))
        c_def[B0,y0] = yd-Bgrid[B0]*(1-m.h)*rb_def[B0,y0]
        vd[B0,y0] = (c_def[B0,y0]^(1-m.σ))/(1-m.σ) + m.α_m*(rb_def[B0,y0]^(1-m.η))/(1-m.η)
        # vd[B0,y0] = U_fn(m, c_def[B0,y0], rb_def[B0,y0])
        Bprime_def[B0,y0] = 0.
        μ_def[B0,y0] = 0.
        q_def[B0, y0] = 1.
    end

    return nothing
end

function update_values_expectations_1!(
        m::Model_GPU, arrays, arrays_rep, arrays_def
    )

    def_policy, v, ev, evd = arrays
    vr, c_rep, rb_rep, Bprime_rep, μ_rep, i_rep, q_rep, qtilde_rep = arrays_rep
    vd, c_def, rb_def, Bprime_def, μ_def, i_def, q_def, qtilde_def = arrays_def

    B0 = (blockIdx().x-1)*blockDim().x + threadIdx().x
    y0 = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if B0 <= m.nB && y0 <= m.ny

        # tid_x, tid_y, bid_x, bid_y = threadIdx().x, threadIdx().y, blockIdx().x, blockIdx().y
        # @cuprintln("y0 $y0, B0 $B0, B1_def_hc $B1_def_hc, tid_x $tid_x, tid_y $tid_y, bid_x $bid_x, bid_y $bid_y")

        if exp((vd[B0,y0]-vr[B0,y0])/m.ρ_δ) == Inf
            def_policy[B0,y0] = 1.
        else
            def_policy[B0,y0] = exp((vd[B0,y0]-vr[B0,y0])/m.ρ_δ) / (1+exp((vd[B0,y0]-vr[B0,y0])/m.ρ_δ))
        end    
        v[B0,y0] = def_policy[B0,y0]*vd[B0,y0] + (1-def_policy[B0,y0])*vr[B0,y0]
    end

    return nothing
end

function update_values_expectations_2!(
        m::Model_GPU, grids, arrays, arrays_rep, arrays_def, eulers_rep, eulers_def
    )

    Bgrid, ygrid, P = grids
    def_policy, v, ev, evd = arrays
    vr, c_rep, rb_rep, Bprime_rep, μ_rep, i_rep, q_rep, qtilde_rep = arrays_rep
    vd, c_def, rb_def, Bprime_def, μ_def, i_def, q_def, qtilde_def = arrays_def
    moneyEulerRHS_rep, bondsEulerRHS_rep = eulers_rep
    moneyEulerRHS_def, bondsEulerRHS_def = eulers_def

    B0 = (blockIdx().x-1)*blockDim().x + threadIdx().x
    y0 = (blockIdx().y-1)*blockDim().y + threadIdx().y

    if B0 <= lastindex(Bgrid) && y0 <= lastindex(ygrid)

        B1 = B0
        B1_def = B1     # to keep things clearer when computing qtilde_def
        B1_def_hc = CUDA.min(CUDA.searchsortedfirst(Bgrid, Bgrid[B1_def]*(1-m.h)), m.nB)
        
        # tid_x, tid_y, bid_x, bid_y = threadIdx().x, threadIdx().y, blockIdx().x, blockIdx().y
        # @cuprintln("y0 $y0, B0 $B0, B1_def_hc $B1_def_hc, tid_x $tid_x, tid_y $tid_y, bid_x $bid_x, bid_y $bid_y")

        #if exp((vd[B0,y0]-vr[B0,y0])/m.ρ_δ) == Inf
        #    def_policy[B0,y0] = 1.
        #else
        #    def_policy[B0,y0] = exp((vd[B0,y0]-vr[B0,y0])/m.ρ_δ) / (1+exp((vd[B0,y0]-vr[B0,y0])/m.ρ_δ))
        #end    
        #v[B0,y0] = def_policy[B0,y0]*vd[B0,y0] + (1-def_policy[B0,y0])*vr[B0,y0]

        # Reset values to zero
        ev[B1, y0], evd[B1, y0] = 0., 0.
        qtilde_rep[B1, y0], qtilde_def[B1, y0] = 0., 0.
        moneyEulerRHS_rep[B1, y0], moneyEulerRHS_def[B1, y0] = 0., 0.
        bondsEulerRHS_rep[B1, y0], bondsEulerRHS_def[B1, y0] = 0., 0.

        for y1 in 1:m.ny

            # compute expected value functions, makes them functions of (̃B',y)
            ev[B1, y0] += v[B1, y1] * P[y0, y1]
            evd[B1, y0] += vd[B1, y1] * P[y0, y1]

            #= 
            the update of the qtilde price fns should only use q_rep and q_def, not qtilde's 
            =#

            # uses: def_policy, q_rep, rb_rep, q_def, rb_def
            qtilde_rep[B1,y0] += (
                (1-def_policy[B1,y1])*(m.λ+(1-m.λ)*(m.κ+q_rep[B1,y1]))*rb_rep[B1,y1] + def_policy[B1,y1]*q_def[B1,y1]*rb_def[B1,y1]
            ) * P[y0,y1]
        
            # uses: q_def, rb_def here, q_rep, rb_rep below
            qtilde_def[B1_def, y0] += (
                (1-m.θ)*q_def[B1_def,y1]*rb_def[B1_def,y1] + 
                m.θ*(
                    (1-def_policy[B1_def_hc,y1])*(m.λ+(1-m.λ)*(m.κ+q_rep[B1_def_hc,y1]))*rb_rep[B1_def_hc,y1] 
                    + def_policy[B1_def_hc,y1]*q_def[B1_def_hc,y1]*rb_def[B1_def_hc,y1]
                )
            ) * P[y0,y1]

            # uses: def_policy, c_rep, rb_rep, c_def, rb_def
            moneyEulerRHS_rep[B1, y0] += m.β_hh * (
                    (1-def_policy[B1,y1])*(c_rep[B1,y1]^(-m.σ) + m.α_m*rb_rep[B1,y1]^(-m.η))*rb_rep[B1,y1] + def_policy[B1,y1]*(c_def[B1,y1]^(-m.σ) + m.α_m*rb_def[B1,y1]^(-m.η))*rb_def[B1,y1]
                ) * P[y0,y1]

            # uses: def_policy, c_rep, rb_rep, c_def, rb_def
            moneyEulerRHS_def[B1, y0] += m.β_hh * (
                    (1-m.θ)*(c_def[B1,y1]^(-m.σ) + m.α_m*rb_def[B1,y1]^(-m.η))*rb_def[B1,y1] + 
                    m.θ*(1-def_policy[B1_def_hc,y1])*(c_rep[B1_def_hc,y1]^(-m.σ) + m.α_m*rb_rep[B1_def_hc,y1]^(-m.η))*rb_rep[B1_def_hc,y1] + 
                    m.θ*def_policy[B1_def_hc,y1]*(c_def[B1_def_hc,y1]^(-m.σ) + m.α_m*rb_def[B1_def_hc,y1]^(-m.η))*rb_def[B1_def_hc,y1]
                ) * P[y0,y1]

            # uses: def_policy, c_rep, rb_rep, c_def, rb_def
            bondsEulerRHS_rep[B1, y0] += m.β_hh * (
                    (1-def_policy[B1,y1])*(c_rep[B1,y1]^(-m.σ))*rb_rep[B1,y1] + 
                    def_policy[B1,y1]*(c_def[B1,y1]^(-m.σ))*rb_def[B1,y1]
                ) * P[y0,y1]

            # uses: def_policy, c_rep, rb_rep, c_def, rb_def
            bondsEulerRHS_def[B1, y0] += m.β_hh * (
                    (1-m.θ)*(c_def[B1,y1]^(-m.σ))*rb_def[B1,y1] + 
                    m.θ*(1-def_policy[B1_def_hc,y1])*(c_rep[B1_def_hc,y1]^(-m.σ))*rb_rep[B1_def_hc,y1] + 
                    m.θ*def_policy[B1_def_hc,y1]*(c_def[B1_def_hc,y1]^(-m.σ))*rb_def[B1_def_hc,y1]
                ) * P[y0,y1]
        end
    end

    return nothing
end

function vr_gridsearch(m::Model_GPU, grids, arrays, arrays_rep, eulers_rep, arrays_temp_rep)
    
    #B0s, y0s = tests
    Bgrid, ygrid, P = grids
    def_policy, v, ev, evd = arrays
    vr, c_rep, rb_rep, Bprime_rep, μ_rep, i_rep, q_rep, qtilde_rep = arrays_rep
    moneyEulerRHS_rep, bondsEulerRHS_rep = eulers_rep
    vs_rep, cs_rep, rbs_rep, μs_rep, qs_rep, is_rep, cps_rep = arrays_temp_rep
    
    B0 = (blockIdx().x-1)*blockDim().x + threadIdx().x
    y0 = (blockIdx().y-1)*blockDim().y + threadIdx().y
    
    # tid_x, tid_y, bid_x, bid_y = threadIdx().x, threadIdx().y, blockIdx().x, blockIdx().y
    # @cuprintln("y0 $y0, B0 $B0, tid_x $tid_x, tid_y $tid_y, bid_x $bid_x, bid_y $bid_y")
    
    # xstar, fstar = my_brent(x->B0*x^2+y0*x, Float32(-10), Float32(10))
    # vs[B0,y0] = fstar
    # cs[B0,y0] = xstar

    if B0 <= lastindex(Bgrid) && y0 <= lastindex(ygrid)

        #y0s[B0,y0]=y0
        #B0s[B0,y0]=B0
        vstar = Float32(-Inf)

        for B1 in 1:m.nB
            c_lb = Float32(1e-4)
            c_ub = ygrid[y0] + qtilde_rep[B1, y0]*Bgrid[B1]

            c, ~ = my_brent(
                        c -> -(c^(1-m.σ))/(1-m.σ) - m.α_m*((
                            (ygrid[y0]+qtilde_rep[B1,y0]*Bgrid[B1]-c)/(Bgrid[B0]*(m.λ+(1-m.λ)*m.κ + qtilde_rep[B1,y0]*(1-m.λ)*c^(-m.σ)/(m.β_hh*moneyEulerRHS_rep[B1, y0])))
                       )^(1-m.η))/(1-m.η), 
                        c_lb, c_ub)
            #c = c_ub/4
            
            rb = (ygrid[y0]+qtilde_rep[B1,y0]*Bgrid[B1]-c)/(Bgrid[B0]*(m.λ+(1-m.λ)*m.κ + qtilde_rep[B1,y0]*(1-m.λ)*c^(-m.σ)/(m.β_hh*moneyEulerRHS_rep[B1, y0])))
            ucprime = c^(-m.σ)
            μ = moneyEulerRHS_rep[B1,y0] / (rb * ucprime) - 1

            cs_rep[B0,y0,B1] = c
            μs_rep[B0,y0,B1] = μ
            rbs_rep[B0,y0,B1] = rb
            qs_rep[B0,y0,B1] = qtilde_rep[B1, y0] / (rb*(1+μ)*(1+m.r))
            is_rep[B0,y0,B1] = ((1+μ)*rb*ucprime) / bondsEulerRHS_rep[B1, y0]
            v = (c^(1-m.σ))/(1-m.σ) + m.α_m*(rb^(1-m.η))/(1-m.η) + m.β * ev[B1, y0]
            vs_rep[B0,y0,B1] = v

            vstar = ifelse(v>vstar, v, vstar)
        end
        
        D = 0.
        for B1 in 1:m.nB
            D += exp((vs_rep[B0,y0,B1]-vstar)/m.ρ_B)
        end
        for B1 in 1:m.nB
            cps_rep[B0,y0,B1] = exp((vs_rep[B0,y0,B1]-vstar)/m.ρ_B)/D
        end
        
        vr[B0,y0] = vstar + m.ρ_B*log(D)

        c_rep[B0,y0], rb_rep[B0,y0], Bprime_rep[B0,y0], q_rep[B0,y0], i_rep[B0,y0], μ_rep[B0,y0] = 0., 0., 0., 0., 0., 0.
        for B1 in 1:m.nB
            cp = cps_rep[B0,y0,B1]
            c_rep[B0,y0] += cp * cs_rep[B0,y0,B1]
            rb_rep[B0,y0] += cp * rbs_rep[B0,y0,B1]
            Bprime_rep[B0,y0] += cp * Bgrid[B1]
            q_rep[B0,y0] += cp * qs_rep[B0,y0,B1]
            i_rep[B0,y0] += cp * is_rep[B0,y0,B1]
            μ_rep[B0,y0] += cp * μs_rep[B0,y0,B1]
        end
    end

    return nothing
end

function vd_gridsearch(m::Model_GPU, grids, arrays, arrays_def, eulers_def, arrays_temp_def)
    
    #B0s, y0s = tests
    Bgrid, ygrid, P = grids
    def_policy, v, ev, evd = arrays
    vd, c_def, rb_def, Bprime_def, μ_def, i_def, q_def, qtilde_def = arrays_def
    moneyEulerRHS_def, bondsEulerRHS_def = eulers_def
    vs_def, cs_def, rbs_def, μs_def, qs_def, is_def, cps_def = arrays_temp_def
    
    B0 = (blockIdx().x-1)*blockDim().x + threadIdx().x
    y0 = (blockIdx().y-1)*blockDim().y + threadIdx().y
    
    # tid_x, tid_y, bid_x, bid_y = threadIdx().x, threadIdx().y, blockIdx().x, blockIdx().y
    # @cuprintln("y0 $y0, B0 $B0, tid_x $tid_x, tid_y $tid_y, bid_x $bid_x, bid_y $bid_y")
    
    # xstar, fstar = my_brent(x->B0*x^2+y0*x, Float32(-10), Float32(10))
    # vs[B0,y0] = fstar
    # cs[B0,y0] = xstar

    if B0 <= lastindex(Bgrid) && y0 <= lastindex(ygrid)

        #y0s[B0,y0]=y0
        #B0s[B0,y0]=B0
        vstar = Float32(-Inf)

        for B1_def in 1:m.nB
            
            c = ydef_fn(m, ygrid[y0])
            μ = Bgrid[B0]/Bgrid[B1_def]-1
            ucprime = c^(-m.σ)
            rb = moneyEulerRHS_def[B1_def, y0] / (ucprime * (1+μ))
            B1_def_hc = min(searchsortedfirst(Bgrid, Bgrid[B1_def]*(1-m.h)), m.nB)
            

            cs_def[B0,y0,B1_def] = c
            μs_def[B0,y0,B1_def] = μ
            rbs_def[B0,y0,B1_def] = rb
            qs_def[B0,y0,B1_def] = qtilde_def[B1_def, y0] / (rb*(1+μ)*(1+m.r))
            is_def[B0,y0,B1_def] = ((1+μ)*rb*ucprime) / bondsEulerRHS_def[B1_def, y0]
            v = (c^(1-m.σ))/(1-m.σ) + m.α_m*(rb^(1-m.η))/(1-m.η) + m.β * (
                    (1-m.θ)*evd[B1_def, y0] + m.θ*ev[B1_def_hc, y0]
                )
            vs_def[B0,y0,B1_def] = v

            vstar = ifelse(v>vstar, v, vstar)
        end
  
        D = 0.
        for B1_def in 1:m.nB
            D += exp((vs_def[B0,y0,B1_def]-vstar)/m.ρ_μ)
        end
        for B1_def in 1:m.nB
            cps_def[B0,y0,B1_def] = exp((vs_def[B0,y0,B1_def]-vstar)/m.ρ_μ) / D
        end

        vd[B0,y0] = vstar + m.ρ_μ*log(D)
        
        c_def[B0,y0], rb_def[B0,y0], Bprime_def[B0,y0], q_def[B0,y0], i_def[B0,y0], μ_def[B0,y0] = 0., 0., 0., 0., 0., 0.
        for B1_def in 1:m.nB
            cp = cps_def[B0,y0,B1_def]
            c_def[B0,y0] += cp * cs_def[B0,y0,B1_def]
            rb_def[B0,y0] += cp * rbs_def[B0,y0,B1_def]
            Bprime_def[B0,y0] += cp * Bgrid[B1_def]
            q_def[B0,y0] += cp * qs_def[B0,y0,B1_def]
            i_def[B0,y0] += cp * is_def[B0,y0,B1_def]
            μ_def[B0,y0] += cp * μs_def[B0,y0,B1_def]
        end
    end

    return nothing
end

function main_gpu(m::Model_GPU; max_iter::Int=1000, print_iter::Int=0, plot::Int=0, ret_arrays::Bool=false, simulate::Bool=false)

    #ts = (cld(m.nB, bs[1]), cld(m.ny, bs[2]))
    threadsize = 16
    ts = (threadsize, threadsize)
    blocksize = cld(m.nB, threadsize)
    bs = (blocksize, blocksize)

    # Bond grid
    Bgrid = collect(range(m.Bgrid_lb, stop=m.Bgrid_ub, length=m.nB))
    Bgrid = CuArray(Bgrid)

    # Endowment grid and transition probs
    grid, P = tauchen_carlo(m.ny, m.ρ, m.σ_ϵ)
    ygrid=CuArray(exp.(grid))
    P = CuArray(P)

    # Initialise arrays
    v0, v, ev, evd, def_policy = [CUDA.zeros(m.nB, m.ny) for i in 1:5]
    vr, c_rep, rb_rep, Bprime_rep, μ_rep, i_rep, q_rep, qtilde_rep, moneyEulerRHS_rep, bondsEulerRHS_rep = [CUDA.zeros(m.nB, m.ny) for i in 1:10]
    vd, c_def, rb_def, Bprime_def, μ_def, i_def, q_def, qtilde_def, moneyEulerRHS_def, bondsEulerRHS_def = [CUDA.zeros(m.nB, m.ny) for i in 1:10]
    q0_rep = CUDA.zeros(m.nB, m.ny)
    vs_rep, cs_rep, rbs_rep, μs_rep, qs_rep, is_rep, cps_rep = [CUDA.zeros(m.nB, m.ny, m.nB) for i in 1:7]
    vs_def, cs_def, rbs_def, μs_def, qs_def, is_def, cps_def = [CUDA.zeros(m.nB, m.ny, m.nB) for i in 1:7]
    B0s, y0s = [CUDA.zeros(m.nB, m.ny) for i in 1:2]


    # Collect arrays in tuples
    grids = (Bgrid, ygrid, P)
    arrays_rep = (vr, c_rep, rb_rep, Bprime_rep, μ_rep, i_rep, q_rep, qtilde_rep)
    arrays_def = (vd, c_def, rb_def, Bprime_def, μ_def, i_def, q_def, qtilde_def)
    arrays = (def_policy, v, ev, evd)
    eulers_rep = (moneyEulerRHS_rep, bondsEulerRHS_rep)
    eulers_def = (moneyEulerRHS_def, bondsEulerRHS_def)
    arrays_temp_rep = (vs_rep, cs_rep, rbs_rep, μs_rep, qs_rep, is_rep, cps_rep)
    arrays_temp_def = (vs_def, cs_def, rbs_def, μs_def, qs_def, is_def, cps_def)
    

    # Initialise everything
    @cuda threads=ts blocks=bs model_init!(m, grids, arrays_rep, arrays_def)
    @cuda threads=ts blocks=bs update_values_expectations_1!(m, arrays, arrays_rep, arrays_def)
    @cuda threads=ts blocks=bs update_values_expectations_2!(m, grids, arrays, arrays_rep, arrays_def, eulers_rep, eulers_def)

    # Iterative process
    err = 1e5
    tol = 1e-5
    iter = 0
    
    while (err > tol) & (iter < max_iter)
        
        iter += 1
        err = 0.
        v0 .= v
        #q0_rep .= q_rep
        #err_qrep = 0.
        #Vd0 = deepcopy(Vd)
        #Price0 = deepcopy(Price)
        # println(vd)
        
        @cuda threads=ts blocks=bs vr_gridsearch(m, grids, arrays, arrays_rep, eulers_rep, arrays_temp_rep)
        @cuda threads=ts blocks=bs vd_gridsearch(m, grids, arrays, arrays_def, eulers_def, arrays_temp_def)
        @cuda threads=ts blocks=bs update_values_expectations_1!(m, arrays, arrays_rep, arrays_def)
        @cuda threads=ts blocks=bs update_values_expectations_2!(m, grids, arrays, arrays_rep, arrays_def, eulers_rep, eulers_def)

        err = maximum(@. abs(v-v0))

        if print_iter > 0
            if rem(iter, print_iter) == 0
                println(@sprintf("iter %.0f, error %.2e", iter, err))
            end
        end
    end

    println(@sprintf("finished with %.0f iters, error %.2e", iter, err))

    # Simulation: allocate and Pre-Fill
    y_sim, B_sim, q_sim, rb_sim, μ_sim, infl_sim, exp_def, exp_infl, c_sim, i_sim = [zeros(m.T_sim) for i in 1:10]
    def_status, def_new = [zeros(Bool, m.T_sim) for i in 1:2]
    B_sim_indices, y_sim_indices = [zeros(Int, m.T_sim) for i in 1:2]
    taste_errors = zeros(3)
    
    arrays_sim = (y_sim, B_sim, q_sim, rb_sim, μ_sim, def_status, def_new, infl_sim, exp_def, exp_infl, c_sim, i_sim, B_sim_indices, y_sim_indices, taste_errors)

    if simulate
        # Simulate the economy
        simulate_cg!(m, grids, arrays, arrays_rep, arrays_def, arrays_temp_rep, arrays_temp_def, arrays_sim)

        println("sim_mom \n")
        @time begin
          # Compute moments
          sim_mom!(m, arrays_sim)
        end
    end

    if plot>0
        plot_eqm(m, grids, arrays, arrays_rep, arrays_def, fn=plot)
    end
    
    if ret_arrays
        return m, grids, arrays, arrays_rep, arrays_def, arrays_temp_rep, arrays_temp_def
    else
        return nothing
    end
end

#= 
SIMULATIONS
=#
function markov_draw!(grid::Vector{Float32}, P::Array{Float32}, out_vector::Vector{Float32}, T::Int32; ret_indices::Bool=false, init_index::Int=round(Int, length(grid)/2), RNG::MersenneTwister=MersenneTwister())
    #=
    Generate a sample path of a Markov chain, from a given initial state. The resulting vector, of dimension T (number of simulation periods), has the state values of the Markov chain as elements.
    Uses: P (the discrete Markov transition matrix), grid (the state grid)
    =#
    Q = cumsum(P, dims=2)   # CDF, for each state
    simulated_indices = zeros(Int32, T)
    simulated_indices[1] = init_index
    randdraws = rand(RNG, T)
    for i in 2:T
        i_prev = simulated_indices[i-1]
        simulated_indices[i] = searchsortedfirst(view(Q,i_prev,:), randdraws[i-1])
    end

    if ret_indices
        out_vector .= simulated_indices
    else
        out_vector .= grid[simulated_indices]
    end
    
    return nothing
end

function simulate_cg!(m::Model_GPU, grids, arrays, arrays_rep, arrays_def, arrays_temp_rep, arrays_temp_def, arrays_sim;
        y_init=1., B_init=1.
    )

    #=
    REMEMBER TO DON'T COUNT DEFAULT PERIODS WHEN COMPUTING SPREADS !!!!
    =#
    # debt_policy, inflation_policy, cons_policy = get_policies(m)
    
    #Bgrid, ygrid, P = grids
    #ygrid_cpu = Vector{Float32}(ygrid)
    #Bgrid_cpu = Vector{Float32}(Bgrid)
    #P_cpu = Array{Float32, 2}(P)

    Bgrid, ygrid, P = convert(NTuple{3, Array}, grids)
    def_policy, v, ev, evd = convert(NTuple{4, Array}, arrays)
    vr, c_rep, rb_rep, Bprime_rep, μ_rep, i_rep, q_rep, qtilde_rep = convert(NTuple{8, Array}, arrays_rep)
    vd, c_def, rb_def, Bprime_def, μ_def, i_def, q_def, qtilde_def = convert(NTuple{8, Array}, arrays_def)
    vs_rep, cs_rep, rbs_rep, μs_rep, qs_rep, is_rep, cps_rep = convert(NTuple{7, Array}, arrays_temp_rep)
    vs_def, cs_def, rbs_def, μs_def, qs_def, is_def, cps_def = convert(NTuple{7, Array}, arrays_temp_def)

    cps_cum_rep = Array{Float32}(accumulate(+, arrays_temp_rep[end], dims=3))
    cps_cum_def = Array{Float32}(accumulate(+, arrays_temp_def[end], dims=3))
    
    y_sim, B_sim, q_sim, rb_sim, μ_sim, def_status, def_new, infl_sim, exp_def, exp_infl, c_sim, i_sim, B_sim_indices, y_sim_indices, taste_errors = arrays_sim

    # Get initial indices
    y_init_index = searchsortedfirst(ygrid, y_init)
    B_init_index = searchsortedfirst(Bgrid, B_init)
    
    # Create a MarkovChain
    MT = MersenneTwister(6687)
    markov_draw!(ygrid, P, y_sim_indices, m.T_sim, ret_indices=true, init_index=y_init_index, RNG=MT)  
    
    # Get taste shocks for both [B'] and [μ']
    tasteshocks = rand(MT, m.T_sim)

    # Get default shocks
    defshocks = rand(MT, m.T_sim)

    # Get re-entry shocks    
    reentryshocks = rand(MT, m.T_sim)

    # Compute expected value of taste shock "errors", i.e. the average (over iB,iy) expected % utility loss, which is =dot(choiceprobs, log(vs/vstar))
    taste_error_B, taste_error_μ, taste_error_δ = 0., 0., 0.
    #taste_error_B = mean([dot(cps_rep[iB,iy,:], log.(vs_rep[iB,iy,:]./maximum(vs_rep[iB,iy,:]))) for iB in 1:m.nB, iy in 1:m.ny])
    #taste_error_μ = mean([dot(cps_def[iB,iy,:], log.(vs_def[iB,iy,:]./maximum(vs_def[iB,iy,:]))) for iB in 1:m.nB, iy in 1:m.ny])
    #taste_error_δ = mean([log(vd[iB,iy]/max(vr[iB,iy],vd[iB,iy]))+1/(1+exp((vd[iB,iy]-vr[iB,iy])/m.ρ_δ))*log(vr[iB,iy]/vd[iB,iy]) for iB in 1:m.nB, iy in 1:m.ny])

    taste_errors .= [taste_error_B; taste_error_μ; taste_error_δ]

    B_sim_indices[1], def_status[1] = B_init_index, false 

    for t=1:m.T_sim-1

        # Get indices of current period
        iyt, iBt = y_sim_indices[t], B_sim_indices[t]
        y_sim[t] = ygrid[iyt]
        B_sim[t] = Bgrid[iBt]
        #show t, iBt, iyt
        #show B_sim

        #=
        [1] If you are NOT in default *from the past*
        =#
        if def_status[t] == false

            # check if you want to default today
            # default_today = m.vr[iBt, iyt] < m.vd[iBt, iyt]
            default_today = defshocks[t] < def_policy[iBt,iyt]

            # [1a] If you DO want to default today
            if default_today
                def_new[t] = true
                # def_status[t] = true    # change status today

            # [1b] If you do NOT default
            else
                # y_sim[t] = m.ygrid[iyt]
                # B_sim[t] = m.Bgrid[iBt]

                #iBtnext_rep = searchsortedfirst(cps_cum_rep[iBt,iyt,:], tasteshocks[t])
                iBtnext_rep = searchsortedfirst(view(cps_cum_rep,iBt,iyt,:), tasteshocks[t])
                
                B_sim_indices[t+1] = iBtnext_rep

                # wrongchoice_ind[t] = iBtnext_rep != findmax(choiceprobs_rep[iBt,iyt])[2]

                c_sim[t] = cs_rep[iBt,iyt,iBtnext_rep]
                rb_sim[t] = rbs_rep[iBt,iyt,iBtnext_rep]
                q_sim[t] = qs_rep[iBt,iyt,iBtnext_rep]
                μ_sim[t] = μs_rep[iBt,iyt,iBtnext_rep]
                i_sim[t] = is_rep[iBt,iyt,iBtnext_rep]

                if q_sim[t] < 1e-4
                    @show defshocks[t], def_policy[iBt,iyt]
                    @show def_status[t], def_new[t]
                    @show t, iBt, iyt, iBtnext_rep
                    error("q_rep = 0")
                end

                if t==1
                    # normalise M/P=1 at t=0
                    infl_sim[t] = 1/rb_sim[t]-1
                else
                    infl_sim[t] = rb_sim[t-1]*(1+μ_sim[t-1])/rb_sim[t]-1
                end

                #= these we don't have at the moment =#
                # exp_def[t] = defprobs_rep[iBt,iyt,iBtnext_rep]
                # exp_infl[t] = expinfs_rep[iBt,iyt,iBtnext_rep]
            end
        end
        # need to close the if and open another one for the change in def_status[t] to be picked up

        #=
        [2] If you are in default
            - from the past (=> did not get settlement option)
            - from the present (=> just defaulted)
        =#
        if def_status[t] == true || def_new[t] == true

            y_sim[t] = ydef_fn(m, ygrid[iyt])     # replaces y_sim[t]

            # index corresponding to B'=B/(1+μ) tomorrow
            #iB1d = searchsortedfirst(cps_cum_def[iBt,iyt,:], tasteshocks[t])
            iB1d = searchsortedfirst(view(cps_cum_def,iBt,iyt,:), tasteshocks[t])
            B1d = Bgrid[iB1d]

            # wrongchoice_ind[t] = iB1d != findmax(choiceprobs_def[iBt,iyt])[2]

            μ_sim[t] = Bgrid[iBt]/Bgrid[iB1d]-1
            c_sim[t] = cs_def[iBt,iyt,iB1d]
            rb_sim[t] = rbs_def[iBt,iyt,iB1d]
            q_sim[t] = qs_def[iBt,iyt,iB1d]
            i_sim[t] = is_def[iBt,iyt,iB1d]

            if t==1
                # normalise M/P=1 at t=0
                infl_sim[t] = 1/rb_sim[t]-1
            else
                infl_sim[t] = rb_sim[t-1]*(1+μ_sim[t-1])/rb_sim[t]-1
            end

            # actual future B' depends on θ
            if reentryshocks[t] < m.θ && def_new[t] == false
                # with prob θ get settlement option tomorrow, when not after a fresh default
                def_status[t+1] = false

                # B_sim_indices[t+1] = min(searchsortedfirst(m.Bgrid, m.Bgrid[iBt]*(1-m.h)/(1+μ_sim[t])), m.nB)
                B_sim_indices[t+1] = min(searchsortedfirst(Bgrid, B1d*(1-m.h)), m.nB)
                # debt carried over to face settlement choice tomorrow

            else    # with prob 1-θ no settlement option tomorrow
                def_status[t+1] = true

                # B_sim_indices[t+1] = min(searchsortedfirst(m.Bgrid, m.Bgrid[iBt]/(1+μ_sim[t])), m.nB)
                B_sim_indices[t+1] = iB1d
            end
        end
    end

    return nothing
end

function sim_mom!(m::Model_GPU, arrays_sim;
        fignum::Int=0, print_table::Bool=false, calib::Bool=false, longrun::Bool=false, longrun_display::Bool=false, ret_vals::Bool=false, ret_moments::Bool=false
    )

    num_arrays = 14
    #y_sim, B_sim, q_sim, rb_sim, μ_sim, def_status, def_new, infl_sim, exp_def, exp_infl, c_sim, i_sim, B_sim_indices, y_sim_indices = [arrays_sim[i][m.T_discard_sim:end] for i in 1:num_arrays]
    taste_errors = arrays_sim[num_arrays+1]
    y_sim, B_sim, q_sim, rb_sim, μ_sim, def_status, def_new, infl_sim, exp_def, exp_infl, c_sim, i_sim, B_sim_indices, y_sim_indices = [CuArray(arrays_sim[i][m.T_discard_sim:end]) for i in 1:num_arrays]

    T = length(y_sim)

    # Preparation
    #repind = [def_status[i]==false && def_new[i]==false for i in 1:T]
    repind = @. def_status==false && def_new==false
    # indicates with 1 periods with access to markets, i.e. where u enter with good credit standing AND you stay in markets by repaying

    defind = @. repind==false
    #repind_chg = repind[2:end]
    T_rep = sum(repind)

    taste_error_B, taste_error_μ, taste_error_δ = [taste_errors[i] for i in 1:3]

    # Total spread (default + inflation). Follows Hatchondo and Martinez's (2009) calculations
    ytms = @. (m.λ+(1-m.λ)*m.κ)/q_sim-m.λ       # constant yield implied by bond prices
    spread = @. ((1+ytms)/(1+m.r))^4-1          # annualised total spread
    #spread_rep = spread[repind]

    # M1/GDP
    #mean_mbasegdp = mean((rb_sim[repind]./y_sim[repind]))
    mean_mbasegdp = mean(@. rb_sim*repind/y_sim)
    # mean_mbasegdp = mean((1 ./(P_sim_val[repind])))
    
    #mean_debtgdp = mean(B_sim[repind].*rb_sim[repind]./y_sim[repind])
    mean_debtgdp = mean(@. B_sim*rb_sim/y_sim)

    #mean_debtmoney_rep = mean(B_sim[repind])
    mean_debtmoney_rep = mean(@. B_sim*repind)
    sd_debtmoney_rep = std(B_sim.*repind)

    mean_debtmoney_def = mean(B_sim.*defind)
    sd_debtmoney_def = std(B_sim.*defind)

    mean_consgdp = mean(@. c_sim*repind/y_sim)
    sd_consgdp = std(@. c_sim*repind/y_sim)

    # mean_spendgdp = mean(g_sim_val[repind]./y_sim[repind])
    # sd_spendgdp = std(g_sim_val[repind]./y_sim[repind])

    # mean_cgratio = mean(c_sim[repind]./g_sim_val[repind])
    # sd_cgratio = std(c_sim[repind]./g_sim_val[repind])

    cpi_annual_rep = (1 .+infl_sim[repind]).^4 .-1
    cpi_annual_def = (1 .+infl_sim[defind]).^4 .-1
    mean_cpi_rep = mean(cpi_annual_rep)     # average cpi YoY
    mean_cpi_def = mean(cpi_annual_def)     # average cpi YoY

    mu_annual_rep = (1 .+μ_sim[repind]).^4 .-1
    mean_mu_rep = mean(mu_annual_rep)
    sd_mu_rep = std(mu_annual_rep)

    mu_annual_def = (1 .+μ_sim[defind]).^4 .-1
    mean_mu_def = mean(mu_annual_def)
    sd_mu_def = std(mu_annual_def)

    seign_annual_rep = mu_annual_rep.*rb_sim[repind]
    mean_seign_rep = mean(seign_annual_rep)
    sd_seign_rep = std(seign_annual_rep)

    seign_stat = mean(seign_annual_rep)/std(seign_annual_rep)

    cds = 1 .-(1 .- exp_def).^4     # default probability really
    cds_rep = cds[repind]
    mean_cds = mean(cds_rep)        # average default prob
    sd_cds = std(cds_rep)           # stdev default prob

    xcs = (1 .+ exp_infl).^4 .-1
    xcs_rep = xcs[repind]
    mean_xcs = mean(xcs_rep)     # average inflation spread

    intrate_rep = (1 .+ i_sim[repind]).^4 .-1
    mean_intrate_rep = mean(intrate_rep)
    sd_intrate_rep = std(intrate_rep)

    intrate_def = (1 .+ i_sim[defind]).^4 .-1
    mean_intrate_def = mean(intrate_def)
    sd_intrate_def = std(intrate_def)

    repind_chg = [repind[i]==true && repind[i-1]==true for i in 2:T]  # 2:T
    cds_chg_r = [cds[i]-cds[i-1] for i in collect(2:1:T)[repind_chg]]
    xcs_chg_r = [xcs[i]-xcs[i-1] for i in collect(2:1:T)[repind_chg]]

    rho_y_cds = cor(y_sim[repind], cds_rep)
    rho_y_xcs = cor(y_sim[repind], xcs_rep)
    rho_y_cpi = cor(y_sim[repind], infl_sim[repind])
    rho_y_b = cor(y_sim[repind], B_sim[repind].*rb_sim[repind])
    rho_cds_xcs = cor(cds_rep, xcs_rep)
    # cor(exp_infl[repind], exp_def[repind])
    rho_cds_xcs_chg = cor(cds_chg_r, xcs_chg_r)

    # Means
    newdefault_freq_qtrly = sum(def_new)/sum(def_status.==false)
    newdefault_freq_yrly = 1-(1-newdefault_freq_qtrly)^4    # yearly default frequency
    indefault = sum(def_status.==true)/T
    mean_spread = mean(spread_rep)        # average quarterly LC spread

    # Standard deviations
    sd_spread = std(spread_rep)
    sd_xcs = std(xcs_rep)
    sd_cpi_rep = std(cpi_annual_rep)
    sd_cpi_def = std(cpi_annual_def)
    sd_debtgdp = std(B_sim[repind].*rb_sim[repind]./y_sim[repind])
    sd_mbasegdp = std(rb_sim[repind] ./ y_sim[repind])

    # Correlations
    rho_y_spread = cor(y_sim[repind], spread_rep)
    rho_b_spread = cor(B_sim[repind], spread_rep)
    rho_y_cpi = cor(y_sim[repind], infl_sim[repind])
    rho_b_cds = cor(B_sim[repind], cds_rep)
    rho_b_xcs = cor(B_sim[repind], xcs_rep)
    rho_cds_cpi = cor(cds_rep, infl_sim[repind])
    
    if longrun
        longrun_moments = [mean_debtgdp;
                            mean_mbasegdp;
                            mean_cpi_rep;
                            mean_cds;
                            # sd_cds;
                            mean_cgratio;
                            mean_i_rep;

                            mean_cpi_def;
                            sd_cpi_rep;
                            sd_cpi_def;
                            mean_seign_rep;
                            mean_mu_rep]
        return longrun_moments
    end

    if longrun_display
        moments_labels = [  "corr(cds, xcs)",
                            "corr(y, cds)",
                            "corr(y, xcs)",
                            "corr(y, cpi)",
                            "corr(cpi, cds)",
                            "corr(y, b)",
                            " ",
                            "avg(b/y) %",
                            "avg(m=1/P) %",
                            "B/M %",
                            "avg(c/g) %",
                            "avg(c/y) %",
                            "",
                            "yearly default freq %",
                            "yearly default periods %",
                            "avg(i_rep) %, ann.ed",
                            "avg(i_def) %, ann.ed",
                            "avg(cds) %, ann.ed",
                            "std(cds) %, ann.ed",
                            "",
                            "avg(seign_rep) %, ann.ed",
                            "avg(mu_r) %, ann.ed",
                            "avg(cpi_rep) %, ann.ed",
                            "std(cpi_rep) %, ann.ed",
                            "avg(mu_d) %, ann.ed",
                            "avg(cpi_def) %, ann.ed",
                            "std(cpi_def) %, ann.ed"]
        moments = [ rho_cds_xcs,
                    rho_y_cds,
                    rho_y_xcs,
                    rho_y_cpi,
                    rho_cds_cpi,
                    rho_y_b,
                    0.,
                    mean_debtgdp*100,
                    mean_mbasegdp*100,
                    mean_debtmoney*100,
                    mean_cgratio*100,
                    mean_consgdp*100,
                    0.,
                    newdefault_freq_yrly*100,
                    indefault*100,
                    mean_i_rep*100,
                    mean_i_def*100,
                    mean_cds*100,
                    sd_cds*100,
                    0.,
                    mean_seign_rep*100,
                    mean_mu_rep*100,
                    mean_cpi_rep*100,
                    sd_cpi_rep*100,
                    mean_mu_def*100,
                    mean_cpi_def*100,
                    sd_cpi_def*100]

        println(@sprintf("beta=%.2f, beta_hh=%.2f, sigma=%.2f, alpha_m=%.2e, eta=%.2f, alpha_nu=%.2e, eta_agg=%.2f, alpha_g=%.2f, gamma=%.2f, rho_B=%.1e, rho_mu=%.1e, rho_D=%.1e, d0=%.3f, d1=%.3f, ny=%.2f, nB=%.2f, B in [%.2f, %.2f]",

        m.beta, m.beta_hh, m.sigma, m.alpha_m, m.eta, m.alpha_nu, m.eta_agg, m.alpha_g, m.gamma, m.rho_B, m.rho_mu, m.rho_D, m.d0, m.d1, m.ny, m.nB, m.Bgrid[1], m.Bgrid[end]))
        print_moments = [@sprintf("%.2f",x) for x in moments]
        pretty_table([moments_labels print_moments])
    end

    if print_table
        # Bound behaviour    
        time_lb = sum(B_sim.==m.Bgrid_lb)
        time_ub = sum(B_sim.==m.Bgrid_ub)
        if time_ub>0 && time_lb>0
            # println("Bgrid: both bounds are binding")
            println(@sprintf("%% time on LB: %.2f, on UB: %.2f", time_lb/T, time_ub/T))
        elseif time_ub>0
            # println("Bgrid: upper bound is binding")
            println(@sprintf("%% time on UB: %.2f", time_ub/T))
        elseif time_lb>0
            # println("Bgrid: lower bound is binding")
            println(@sprintf("%% time on LB: %.2f", time_lb/T))
        end
        
        # Print out tables
        column_headers = ["statistic", "corr, mean", "stdev"]
        output = [
            "corr(cds, xcs)"           rho_cds_xcs        "";
            "corr(cds_chg, xcs_chg)"   rho_cds_xcs_chg    "";
            "corr(y, spread)"          rho_y_spread       "";
            "corr(y, cds)"             rho_y_cds          "";
            "corr(y, xcs)"             rho_y_xcs          "";
            "corr(y, cpi)"             rho_y_cpi          "";
            "corr(cpi, cds)"           rho_cds_cpi        "";
            "corr(y, b)"               rho_y_b            "";
            "___"                      ""                 "";
            "taste error B/M"          taste_error_B      "";
            "taste error mu"           taste_error_μ      "";
            "taste error default"      taste_error_δ      "";
            "___"                      ""                 "";
            "b/y (%)"  mean_debtgdp*100   sd_debtgdp*100;
            "m/y (%)"  mean_mbasegdp*100  sd_mbasegdp*100;
            "B/M_rep (%)"  mean_debtmoney_rep*100    sd_debtmoney_rep*100;
            "B/M_def (%)"  mean_debtmoney_def*100    sd_debtmoney_def*100;
            "c/y (%)"   mean_consgdp*100   sd_consgdp*100;
            # "g/y (%)"   mean_spendgdp*100  sd_spendgdp*100;
            # "c/g (%)"   mean_cgratio*100   sd_cgratio*100;
            "LC spread (%), ann.ed" mean_spread*100 sd_spread*100;
            "yearly default freq %" newdefault_freq_yrly*100 "";
            "yearly default periods %" indefault*100 "";
            "intrate_rep (%), ann.ed" mean_intrate_rep*100 sd_intrate_rep*100;
            "intrate_def (%), ann.ed" mean_intrate_def*100 sd_intrate_def*100;
            "cds (%), ann.ed" mean_cds*100 sd_cds*100;
            "xcs (%), ann.ed" mean_xcs*100 sd_xcs*100;
            "seign_rep (%), ann.ed" mean_seign_rep*100 sd_seign_rep*100;
            "mu_rep (%), ann.ed" mean_mu_rep*100 sd_mu_rep*100;
            "cpi_rep (%), ann.ed" mean_cpi_rep*100 sd_cpi_rep*100;
            "mu_def (%), ann.ed" mean_mu_def*100 sd_mu_def*100;
            "cpi_def (%), ann.ed" mean_cpi_def*100 sd_cpi_def*100
            ]

        println(@sprintf("ny=%.0f, nB=%.0f, B∈[%.2f, %.2f], θ=%.2f, h=%.2f, ρ_B=%.1e, ρ_mu=%.1e, ρ_D=%.1e, β_hh=%.4f, r=%.4f, σ=%.0f, η=%.1f, α_G=%.4f, γ=%.0f, λ=%.4f, κ=%.4f, β=%.2f, α_M=%.1e, d0=%.4f, d1=%.4f",
        m.ny, m.nB, m.Bgrid_lb, m.Bgrid_ub, m.θ, m.h,
        m.ρ_B, m.ρ_μ, m.ρ_δ,
        m.β_hh, m.r, m.σ, m.η, m.α_g,
        m.γ, m.λ, m.κ, 
        m.β, m.α_m, m.d0, m.d1))

        # pars_labels = ["beta", "beta_hh", "sigma", "alpha_m", "eta", "gamma", "g_lb", "r", "rho", "stdev", "theta", "h", "d0", "d1", "ny", "nB", "B_lb", "B_ub"]
        # pars = [m.beta, m.beta_hh, m.sigma, m.alpha_m, m.eta, m.gamma, m.g_lb, m.r, m.rho, m.stdev, m.theta, m.h, m.d0, m.d1, m.ny, m.nB, m.Bgrid[1], m.Bgrid[end]]

        function formatfn(v, row, col)
            if isa(v, String)
                return v
            else
                if abs(v) < 10^5 && abs(v) >= 1e-2
                    return @sprintf("%.2f", v)
                else
                    return @sprintf("%.1e", v)
                end
            end
        end
        pretty_table(output, header=column_headers, formatters=formatfn)
        # println(@sprintf("Time on upper bgrid bound = %.4f %%\nTime on lower bgrid bound = %.4f %%\nTime on upper pigrid bound = %.4f %%", sum(B_sim[repind].==m.bgrid[end])/sum(repind)*100, sum(B_sim[repind].==m.bgrid[1])/sum(repind)*100, sum(infl_sim[repind].==0.2)/sum(repind)*100))

        if sum(cds_rep)==0
            println("Default probs are always zero!")
        end
    end

    if ret_vals
        return y_sim, B_sim, q_sim, P_sim_val, μ_sim, def_status, def_new, infl_sim, exp_def, exp_infl, c_sim
    elseif ret_moments
        return pars_labels, print_pars, moments_labels, print_moments
    else
        return nothing
    end
end

#=
PLOTS
=#
function plot_eqm(m, grids, arrays, arrays_rep, arrays_def; fn::Int=1)

    figure(fn)
    clf()
    rows = 3; cols = 3
    fig, axs = subplots(rows, cols, num=fn, clear=true)

    Bgrid, ygrid, P = grids
    def_policy, v, ev, evd = arrays
    vr, c_rep, rb_rep, Bprime_rep, μ_rep, i_rep, q_rep, qtilde_rep = arrays_rep
    vd, c_def, rb_def, Bprime_def, μ_def, i_def, q_def, qtilde_def = arrays_def
    
    nB, ny = size(v)

    unc_stdev = m.σ_ϵ/sqrt((1-m.ρ^2))
    iys = [min(searchsortedfirst(ygrid, 1+unc_stdev*j),ny) for j in -2:1:2]
    # iys = [5;10;15]
   
    for iy in iys

        sca(axs[1]); title(L"V(\tilde B,y)")
        l1,=plot(Bgrid, vr[:,iy], label=@sprintf("y = %.4f", ygrid[iy]))
        c=l1.get_color()
        plot(Bgrid, vd[:,iy], "--", color=c)
        xlabel(L"\tilde B")

        sca(axs[2]); title(L"m(\tilde B,y)")
        plot(Bgrid, rb_rep[:,iy], "-", color=c)
        plot(Bgrid, rb_def[:,iy], "--", color=c)
        xlabel(L"\tilde B")

        sca(axs[3]); title("Consumption")
        plot(Bgrid, c_rep[:,iy])
        plot(Bgrid, c_def[:,iy], "--", color=c)
        xlabel(L"\tilde B")

        sca(axs[4]); title(L"\tilde B'(\tilde B, y)")
        plot(Bgrid, Bprime_rep[:,iy], "-", color=c)
        plot(Bgrid, Bprime_def[:,iy], "--", color=c)
        xlabel(L"\tilde B")
        
        sca(axs[5]); title(L"\mu(\tilde B, y)")
        plot(Bgrid, μ_rep[:,iy], "-", color=c)
        plot(Bgrid, μ_def[:,iy], "--", color=c)
        plot(Bgrid, zeros(nB), ":k")
        xlabel(L"\tilde B")

        sca(axs[6]); title(L"\tilde B/(1+\mu')")
        plot(Bgrid, Bgrid./(1 .+μ_def[:,iy]), "--", color=c)
        plot(Bgrid, Bgrid, "--k")
        xlabel(L"\tilde B")

        sca(axs[7]); title("Eqm Debt Prices")
        plot(Bgrid, q_rep[:,iy], "-", color=c)
        plot(Bgrid, q_def[:,iy], "--", color=c)
        xlabel(L"\tilde B")

        sca(axs[8]); title("eqm default probs")
        plot(Bgrid, def_policy[:,iy], "-", color=c)
        xlabel(L"\tilde B")
    end

    fs_base = 6
    plot_style(fn, fs=fs_base, fs_legend=fs_base-2, fs_label=fs_base, format_tick_labels=false,  yticklabsize=fs_base-2, xticklabsize=fs_base-2, tick_direction="out", tick_pad=2, tick_width=0.5, tick_length=3, hl=2, legend_loc=0, xlabel_coords=(0.5, -0.05), ylabel_coords=(-0.05,0.5))
    subplots_adjust(left=0.05, right=0.95, top=0.94, bottom=0.06, wspace=0.3, hspace=0.3)
end

function plot_style(fignum; 
	fs::Real=10, 				# fontsize singlegraph
	fs_title::Real=NaN,   				# fontsize title (defaults to fs)
	fs_legend::Real=NaN, 				# fontsize legend (defaults to fs)
	fs_label::Real=NaN, 				# fontsize label (defaults to fs)
	xticklabsize::Real=NaN, 	
	yticklabsize::Real=NaN, 	
	xlabel_coords::Tuple=(0.5, -0.1),
	ylabel_coords::Tuple=(-0.1, 0.5),
	title_coords::Tuple=(0.5, 1.05),
	legend_labelspacing::Real=0,	# vertical space between legend labels
	legend_columnspacing::Real=2, 					# spacing between columns
	legend_handlepad::Real=1,				# space between legend handle and text
	legend_loc=0,						# legend location (0=best)
	hl::Real=1, 					# legend handlelength
	legend_ncol=1,
	ldir_subplot::String="in",
	xtick_label_format::String="%.2f",
	ytick_label_format::String="%.2f", 
	xtick_number=NaN,
	ytick_number=NaN, 
	format_tick_labels::Bool=false,
	tick_direction="in",
	tick_rotation=0,
	xtick_rotation=NaN,
	ytick_rotation=NaN,
	tick_pad=4,
	tick_width=1,
	tick_length=4
	)

	fig = Float64	
	if isa(fignum,PyPlot.Figure)
		fig = fignum
	elseif isa(fignum,Int)
		fig = figure(fignum)
	end
	axs = fig.axes
	fmt_x = matplotlib.ticker.FormatStrFormatter(xtick_label_format)
	fmt_y = matplotlib.ticker.FormatStrFormatter(ytick_label_format)

	if isnan(fs_legend); fs_legend=fs; end
	if isnan(fs_title); fs_title=fs; end
	if isnan(fs_label); fs_label=fs; end
	if isnan(xticklabsize); xticklabsize=fs-4; end
	if isnan(yticklabsize); yticklabsize=fs-4; end

	for i in 1:length(axs)
		ca = axs[i]
		
		ca.tick_params.("both", labelcolor = "k", direction = tick_direction, length = tick_length, width = tick_width, pad = tick_pad, colors = "k", rotation=tick_rotation)
		ca.tick_params.("x", labelsize = xticklabsize, rotation=(isnan(xtick_rotation) ? tick_rotation : xtick_rotation))
		ca.tick_params.("y", labelsize = yticklabsize, rotation=(isnan(ytick_rotation) ? tick_rotation : ytick_rotation))
		ca.xaxis.label.set_size(fs_label)
		ca.yaxis.label.set_size(fs_label)
		ca.xaxis.set_label_coords.(xlabel_coords[1], xlabel_coords[2])	    
		ca.yaxis.set_label_coords.(ylabel_coords[1], ylabel_coords[2])

		if format_tick_labels
			ca.xaxis.set_major_formatter(fmt_x)
			ca.yaxis.set_major_formatter(fmt_y)
		end
		
		if length(ca.get_title())>0 		# there is a title
			ca.title.set_size(fs_title)
			ca.title.set_y.(title_coords)
		end

		if isnan(xtick_number)==false
			xl, xh = ca.get_xlim()
			ca.xaxis.set_ticks(range(xl, stop=xh, length=xtick_number))
		end
		if isnan(ytick_number)==false
			yl, yh = ca.get_ylim()
			ca.yaxis.set_ticks(range(yl, stop=yh, length=ytick_number))
		end

		
		if isa(ca.get_legend_handles_labels()[2],Array{String})	# there is a legend			
			ca.legend(fontsize = fs_legend, handlelength = hl, frameon = false, labelspacing=legend_labelspacing, loc=legend_loc, ncol=legend_ncol, columnspacing=legend_columnspacing, handletextpad=legend_handlepad)
		end
	end
end

plot_style (generic function with 1 method)

In [294]:
@time main_gpu(Model_GPU(nB=51, ny=21, T_sim=10^6, T_discard_sim=100), max_iter=100, print_iter=100, simulate=true)

finished with 62 iters, error 9.54e-06
sim_mom 

  0.048821 seconds (641 allocations: 93.483 MiB)
  0.485500 seconds (16.77 k allocations: 230.380 MiB)


# Execute code

In [107]:
@time main_gpu(Model_GPU(nB=51, ny=21), max_iter=100, print_iter=100)

finished with 62 iters, error 9.54e-06
  0.289952 seconds (15.79 k allocations: 1.932 MiB)


# Debug various things

Let's try to make the simulation faster.

In [297]:
# Find best way to take a mean. Vector multiplication and sum is better than indexing!

function f1(A)
    inds = @. A>0.5
    mean(A[inds])
    return
end

function f2(A)
    inds = @. A>0.5
    mean(A[inds])
    return
end

function f3(A)
    inds = @. A>0.5
    reduce(+, @. A*inds)/reduce(+, inds)
    return
end

function f4(A)
    inds = @. A>0.5
    sum(A.*inds)/sum(inds)
    return
end

function f5(A)
    inds = @. A>0.5
    mean(A.*inds)
    return
end

x = rand(10^6)
y = CuArray(x)

f1(y), f2(y), f3(y), f4(y), f5(y)

(nothing, nothing, nothing, nothing, nothing)

In [298]:
@btime f1($y)
@btime f2($y)
@btime f3($y)
@btime f4($y)
@btime f5($y)

  479.952 μs (260 allocations: 12.20 KiB)
  479.279 μs (260 allocations: 12.20 KiB)
  213.341 μs (112 allocations: 5.66 KiB)
  214.231 μs (112 allocations: 5.66 KiB)
  171.884 μs (72 allocations: 3.56 KiB)


In [10]:
# This shows that indexing CuArrays is tragically slow and should be avoided at all costs
function test(X)
    t = 0.
    for i in 1:lastindex(X)
      t += X[i]
    end
    return t
end

A = rand(Float32, 10^6)
B = CuArray(A)
nothing

In [12]:
@btime test($A)
@btime test($B)

  1.259 ms (1 allocation: 16 bytes)
  10.105 s (3000001 allocations: 457.76 MiB)


499931.26923942566

In [11]:
# This shows that different methods to copy a CuArray into a new Array are identical
function test1(X::CuArray)
    Y = Vector{Float32}(X)
    return
end

function test2(X::CuArray)
    Y = convert(Vector{Float32}, X)
    return
end

function test3(X::CuArray)
    Y = Vector{Float32}(undef, length(X))
    copyto!(Y, X)
    return
end

A = CuArray(rand(Float32, 10^6))

@btime test1($A)
@btime test2($A)
@btime test3($A)

  3.356 ms (5 allocations: 11.44 MiB)
  3.333 ms (5 allocations: 11.44 MiB)
  3.359 ms (5 allocations: 11.44 MiB)


In [42]:
A = CuArray(rand(Float32, 10^6))
B = CuArray(rand(Float32, 10^6))
tup_gpu = A,B
typeof(tup_gpu)
isa(tup_gpu, Tuple{CuArray, CuArray})
tup_cpu = convert(Tuple{Array, Array}, tup_gpu)
[tup_cpu[1] tup_cpu[2]]

C, D = convert(Tuple{Array, Array}, tup_gpu)

typeof(tup_gpu)
isa(tup_gpu, NTuple{2, CuArray})

true